<a href="https://colab.research.google.com/github/vadhri/ai-notebook/blob/main/Agents/llama_index_workflows.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Workflows in LlamaIndex


This notebook is part of the [Hugging Face Agents Course](https://www.hf.co/learn/agents-course), a free Course from beginner to expert, where you learn to build Agents.

![Agents course share](https://huggingface.co/datasets/agents-course/course-images/resolve/main/en/communication/share.png)

## Let's install the dependencies

We will install the dependencies for this unit.

In [1]:
!pip install llama-index datasets llama-index-callbacks-arize-phoenix llama-index-vector-stores-chroma llama-index-utils-workflow llama-index-llms-huggingface-api pyvis -U -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.0/756.0 kB 45.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 90.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.2/295.2 kB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 42.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 68.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 41.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.4/40.4 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.6 MB/s et

And, let's log in to Hugging Face to use serverless Inference APIs.

In [2]:
from huggingface_hub import login

login()

## Basic Workflow Creation

We can start by creating a simple workflow. We use the `StartEvent` and `StopEvent` classes to define the start and stop of the workflow.

In [3]:
from llama_index.core.workflow import StartEvent, StopEvent, Workflow, step


class MyWorkflow(Workflow):
    @step
    async def my_step(self, ev: StartEvent) -> StopEvent:
        # do something here
        return StopEvent(result="Hello, world!")


w = MyWorkflow(timeout=10, verbose=False)
result = await w.run()
result

'Hello, world!'

## Connecting Multiple Steps

We can also create multi-step workflows. Here we pass the event information between steps. Note that we can use type hinting to specify the event type and the flow of the workflow.

In [4]:
from llama_index.core.workflow import Event


class ProcessingEvent(Event):
    intermediate_result: str


class MultiStepWorkflow(Workflow):
    @step
    async def step_one(self, ev: StartEvent) -> ProcessingEvent:
        # Process initial data
        return ProcessingEvent(intermediate_result="Step 1 complete")

    @step
    async def step_two(self, ev: ProcessingEvent) -> StopEvent:
        # Use the intermediate result
        final_result = f"Finished processing: {ev.intermediate_result}"
        return StopEvent(result=final_result)


w = MultiStepWorkflow(timeout=10, verbose=False)
result = await w.run()
result

'Finished processing: Step 1 complete'

## Loops and Branches

We can also use type hinting to create branches and loops. Note that we can use the `|` operator to specify that the step can return multiple types.

In [72]:
from llama_index.core.workflow import Event, Workflow, StartEvent, StopEvent, step
import random

class ProcessingEvent(Event):
    intermediate_result: str

class LoopEvent(Event):
    loop_output: str

class MultiStepWorkflow(Workflow):
    @step
    async def step_one(self, ev: StartEvent) -> ProcessingEvent | LoopEvent:
        if random.randint(0, 1) == 0:
            print("Bad thing happened")
            return LoopEvent(loop_output="Back to step one.")
        else:
            print("Good thing happened")
            return ProcessingEvent(intermediate_result="First step complete.")

    @step
    async def step_two(self, ev: ProcessingEvent | LoopEvent) -> StopEvent:
        if isinstance(ev, ProcessingEvent):
            final_result = f"Finished processing: {ev.intermediate_result}"
            return StopEvent(result=final_result)
        elif isinstance(ev, LoopEvent):
            final_result = f"Loop triggered: {ev.loop_output}"
            return StopEvent(result=final_result)

w = MultiStepWorkflow(verbose=False)
result = await w.run()
result

'Loop triggered: Back to step one.'

## Drawing Workflows

We can also draw workflows using the `draw_all_possible_flows` function.


In [73]:
from llama_index.utils.workflow import draw_all_possible_flows

draw_all_possible_flows(w)

![drawing](https://huggingface.co/datasets/agents-course/course-images/resolve/main/en/unit2/llama-index/workflow-draw.png)

### State Management

Instead of passing the event information between steps, we can use the `Context` type hint to pass information between steps.
This might be useful for long running workflows, where you want to store information between steps.

In [85]:
from llama_index.core.workflow import Event, Context
from llama_index.core.agent.workflow import ReActAgent


class ProcessingEvent(Event):
    intermediate_result: str


class MultiStepWorkflow(Workflow):
    @step
    async def step_one(self, ev: StartEvent, ctx: Context) -> ProcessingEvent:
        # Process initial data
        await ctx.set("query", "What is the capital of France?")
        return ProcessingEvent(intermediate_result="Step 1 complete")

    @step
    async def step_two(self, ev: ProcessingEvent, ctx: Context) -> StopEvent:
        # Use the intermediate result
        query = await ctx.get("query")
        print(f"Query: {query}")
        final_result = f"Finished processing: {ev.intermediate_result}"
        return StopEvent(result=final_result)


w = MultiStepWorkflow(timeout=10, verbose=False)
result = await w.run()
result

'Finished processing: Step 1 complete'

## Multi-Agent Workflows

We can also create multi-agent workflows. Here we define two agents, one that multiplies two integers and one that adds two integers.

In [103]:
from llama_index.core.workflow import Workflow, Event, StartEvent, StopEvent, step
from llama_index.llms.huggingface_api import HuggingFaceInferenceAPI
from llama_index.core.agent import ReActAgent
from llama_index.core.tools import FunctionTool
import random

# Define some tools and wrap them with FunctionTool
def add(a: int, b: int) -> int:
    """Add two numbers."""
    return a + b

def multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b

# Convert functions to FunctionTool objects
add_tool = FunctionTool.from_defaults(fn=add)
multiply_tool = FunctionTool.from_defaults(fn=multiply)

# Initialize the LLM
llm = HuggingFaceInferenceAPI(model_name="Qwen/Qwen2.5-Coder-32B-Instruct")

# Define the agents with FunctionTool objects
multiply_agent = ReActAgent.from_tools(
    tools=[multiply_tool],
    llm=llm,
    verbose=True,
    system_prompt="A helpful assistant that can use a tool to multiply numbers.",
)

addition_agent = ReActAgent.from_tools(
    tools=[add_tool],
    llm=llm,
    verbose=True,
    system_prompt="A helpful assistant that can use a tool to add numbers.",
)

# Define a custom event for agent responses
class AgentResponseEvent(Event):
    response: str

# Define the custom workflow
class MultiAgentWorkflow(Workflow):
    def __init__(self, agents_with_names, root_agent_name, **kwargs):
        super().__init__(**kwargs)
        # Store agents as a dict with explicit names
        self.agents = {name: agent for name, agent in agents_with_names}
        self.root_agent_name = root_agent_name

    @step
    async def route_request(self, ev: StartEvent) -> AgentResponseEvent:
        user_msg = ev.get("user_msg")  # Get the user message from StartEvent
        if not user_msg:
            return AgentResponseEvent(response="No input provided.")

        # Simple routing logic: check the message to decide which agent to use
        if "multiply" in user_msg.lower():
            agent = self.agents["multiply_agent"]
        elif "add" in user_msg.lower():
            agent = self.agents["add_agent"]
        else:
            agent = self.agents[self.root_agent_name]  # Default to root agent

        # Run the selected agent (no await, since chat() is synchronous)
        response = agent.chat(user_msg)
        return AgentResponseEvent(response=str(response))

    @step
    async def finalize(self, ev: AgentResponseEvent) -> StopEvent:
        return StopEvent(result=ev.response)

# Create and run the workflow
workflow = MultiAgentWorkflow(
    agents_with_names=[
        ("multiply_agent", multiply_agent),
        ("add_agent", addition_agent)
    ],
    root_agent_name="multiply_agent",
    verbose=True
)

# Run the system
response = await workflow.run(user_msg="Can you add 1583823 and 98723183723?")  # Note: run() might need await
response

'98724767546'

In [104]:
1583823 + 98723183723 == 98724767546

True